In [13]:
import nltk
import matplotlib.pyplot as plt
import numpy
from nltk.corpus import twitter_samples
import random
import numpy as np
import math as m

nltk.download('twitter_samples')
nltk.download('stopwords')

[nltk_data] Downloading package twitter_samples to
[nltk_data]     /Users/adithyashanker/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/adithyashanker/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
# use predefined function ‘fileids()’ to see the content
twitter_samples.fileids()
# select the set of positive and negative tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

In [15]:
# print positive in greeen
print('\033[92m' + all_positive_tweets[random.randint(0,5000)])

# print negative in red
print('\033[91m' + all_negative_tweets[random.randint(0,5000)])

@JaredLeto Wheeeeeeeeeeeeeeeen?!?!?! :D
I wish I could speak Igbo :(


In [16]:

from nltk.corpus import stopwords          # module for stop words that come with NLTK
from nltk.stem import PorterStemmer        # module for stemming
from nltk.tokenize import TweetTokenizer   # module for tokenizing strings
import re
import string                              # for string operations

stop_words = stopwords.words('english')
punctuation = string.punctuation
tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)

tweets = all_positive_tweets + all_negative_tweets ## Concatenate the lists. 
labels = np.append(np.ones((len(all_positive_tweets),1)), np.zeros((len(all_negative_tweets),1)), axis = 0)

train_pos  = all_positive_tweets[:4000]
train_neg  = all_negative_tweets[:4000]
train_x = train_pos + train_neg 
train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)), axis=0).reshape(-1)


test_pos  = all_positive_tweets[1000:]
test_neg  = all_negative_tweets[1000:]
test_x = test_pos + test_neg
test_y = np.append(np.ones(len(test_pos)),np.zeros(len(test_neg)), axis=0)



In [17]:
print(train_y)

[1. 1. 1. ... 0. 0. 0.]


In [18]:
def preprocess(tweet) :
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tweet = re.sub(r'https?://[^\s\n\r]+', '', tweet)
    tweet = re.sub(r'#', '', tweet)
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    stemmer = PorterStemmer()
    tweet_tokens = tokenizer.tokenize(tweet)
    cleaned_stemmed_tweet = []
    for token in tweet_tokens:
        if token not in stop_words and token not in punctuation :
            cleaned_stemmed_tweet.append(stemmer.stem(token))
    return cleaned_stemmed_tweet



In [19]:


def buildFrequencyTable(features_x,labels_list):
    dictionary = {}
    sum_pos = 0
    sum_neg = 0
    for i in range(len(features_x)) :
        tokens = preprocess(features_x[i])
        for token in tokens:
            if token not in dictionary:
                if labels_list[i] == 1:
                    dictionary[token] = (1,0)
                    sum_pos+=1
                else:
                    dictionary[token] = (0,1)
                    sum_neg+=1
            else:
                if labels_list[i] == 1:
                    dictionary[token] = (dictionary[token][0]+1,dictionary[token][1])
                    sum_pos+=1
                else:
                    dictionary[token] = (dictionary[token][0], dictionary[token][1]+1)
                    sum_neg+=1
    return dictionary,sum_pos,sum_neg
                


In [20]:
def laplace_smoothing(dictionary,sum_pos, sum_neg, alpha):
    for key in dictionary:
        dictionary[key] = (dictionary[key][0]+alpha)/(sum_pos+len(dictionary)) , (dictionary[key][1]+alpha)/(sum_neg+len(dictionary))
    return dictionary

In [21]:


def compute_lambdas(dictionary) :
    for key in dictionary:
        dictionary[key] = (dictionary[key][0], dictionary[key][1], np.log(dictionary[key][0]/dictionary[key][1]))
    return dictionary
        



In [31]:
def predict(dictionary, logprior, tweet) :
    tokens = preprocess(tweet)
    prediction = 0
    for token in tokens :
        if token in dictionary :
            prediction += dictionary[token][2]
    return int((prediction+logprior)>0)


In [34]:
dictionary,sum_pos,sum_neg = buildFrequencyTable(train_x,train_y)
dictionary = laplace_smoothing(dictionary,sum_pos,sum_neg,1)
dictionary = compute_lambdas(dictionary)
predictions = []




In [35]:
for tweet in test_x:
    predictions.append(predict(dictionary, 0, tweet))

print(np.sum(predictions==test_y)/len(test_x))

0.998375
